### 1. IMPORTACIÓN DE LIBRERÍAS

In [4]:
import pandas as pd
import numpy as np

### 2. CARGA DE LOS DATOS (DE MOMENTO CONSIDERANDO UN .CSV)

In [5]:
def load_data(file_path: str) -> pd.DataFrame | None:
    """
    Carga un conjunto de datos desde una ruta de archivo CSV.

    Esta función encapsula la lógica de lectura de datos con Pandas,
    incluyendo un manejo básico de errores si el archivo no se encuentra.

    Args:
        file_path (str): La ruta al archivo .csv que se va a cargar.

    Returns:
        pd.DataFrame | None: Un DataFrame de Pandas con los datos cargados,
        o None si ocurre un error (ej. archivo no encontrado).
    """
    try:
        # Intenta leer el archivo CSV y lo carga en un DataFrame
        df = pd.read_csv(file_path)
        print(f"Datos cargados exitosamente desde: {file_path}")
        return df
    except FileNotFoundError:
        # Manejo de error si el archivo no existe en la ruta especificada
        print(f"Error: El archivo no fue encontrado en la ruta: {file_path}")
        return None
    except Exception as e:
        # Manejo de otros posibles errores durante la carga
        print(f"Ocurrió un error inesperado al cargar el archivo: {e}")
        return None

### 3. DIAGNÓSTICO INICIAL

In [6]:
def get_data_overview(df: pd.DataFrame, df_name: str = "DataFrame") -> None:
    """
    Imprime un resumen completo y diagnóstico de un DataFrame.

    Incluye dimensiones, tipos de datos, estadísticas descriptivas,
    conteo de duplicados y porcentaje de valores nulos.

    Args:
        df (pd.DataFrame): El DataFrame que se va a analizar.
        df_name (str): Un nombre opcional para el DataFrame que se mostrará en los reportes.
    """
    print(f"\n===============ANÁLISIS EXPLORATORIO RÁPIDO PARA: '{df_name}'===============")

    # 1. Dimensiones del DataFrame
    print(f"\n**Dimensiones:** {df.shape[0]} filas y {df.shape[1]} columnas.")

    # 2. Tipos de datos y valores no nulos
    print("\n**Tipos de Datos y Valores No Nulos:**")
    df.info()

    # 3. Estadísticas Descriptivas para variables numéricas
    print("\n**Estadísticas Descriptivas (Numéricas):**")
    # Usamos .T para transponer la tabla y hacerla más legible
    print(df.describe().T)

    # 4. Conteo de filas duplicadas
    duplicates = df.duplicated().sum()
    print(f"\n**Filas Duplicadas:** {duplicates} filas duplicadas encontradas.")

    # 5. Porcentaje de Valores Nulos por columna
    null_percentage = (df.isnull().sum() / len(df)) * 100
    null_info = null_percentage[null_percentage > 0].sort_values(ascending=False)
    
    if not null_info.empty:
        print("\n**Porcentaje de Valores Nulos (>0%):**")
        print(null_info)
    else:
        print("\n**No se encontraron valores nulos.**")
    
    print("\nFin del análisis.")

### 4. LIMPIEZA DE LOS DATOS

##### Validación del esquema. Eliminación de columnas no identificadas

In [7]:
def drop_unnecessary_columns(df: pd.DataFrame, valid_columns: list) -> pd.DataFrame:
    """
    Elimina las columnas de un DataFrame que no están en una lista de columnas válidas.

    Esta función es útil para asegurar que el DataFrame solo contenga las columnas
    esperadas según el esquema definido.

    Args:
        df (pd.DataFrame): El DataFrame a limpiar.
        valid_columns (list): Una lista de strings con los nombres de las
                              columnas que deben permanecer.

    Returns:
        pd.DataFrame: Un nuevo DataFrame que solo contiene las columnas válidas.
    """

    # Identifica las columnas actuales del DataFrame
    current_columns = df.columns.tolist()
    
    # Encuentra las columnas que están en el DataFrame pero no en la lista de válidas
    cols_to_drop = [col for col in current_columns if col not in valid_columns]

    if cols_to_drop:
        print(f"\nColumnas a eliminar: {cols_to_drop}")
        # Elimina las columnas identificadas y devuelve una copia del df modificado
        df_cleaned = df.drop(columns=cols_to_drop)
        print("Columnas innecesarias eliminadas.")
    else:
        print("No se encontraron columnas innecesarias. El esquema es correcto.")
        df_cleaned = df.copy() # Devuelve una copia para mantener la consistencia

    return df_cleaned